# Start code

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from sklearn.model_selection import train_test_split
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
import json
import cv2
import numpy as np
import os
from PIL import Image  # Add this import

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

False


In [7]:
# Read the video
methodstring = 'dl'
read_filename = 'straight_backview_decent.MOV'
video = cv2.VideoCapture(f'input_videos/recorded/{read_filename}')

# Load image
# Set the video capture's position
fps = video.get(cv2.CAP_PROP_FPS)
index_time = 5
index = int(index_time * fps)  # frame index
video.set(cv2.CAP_PROP_POS_FRAMES, index)

# Read the frame at the current position
ret, frame = video.read()
save_filename = f'{methodstring}frame_{index}_{read_filename}.png'
if save_filename not in os.listdir('input_videos/frames/'):
    cv2.imwrite(f'input_videos/frames/{save_filename}', frame)
    image = cv2.imread(f'input_videos/frames/{save_filename}')
else:
    image = cv2.imread(f'input_videos/frames/{save_filename}')




# Create Torch Dataset

In [8]:
 def preprocess_image(image):
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # Apply Canny edge detection
        edges = cv2.Canny(blurred, 50, 150)
        # Create a white mask
        mask = np.ones_like(edges) * 255
        # Apply the mask to the edges
        preprocessed_image = cv2.bitwise_and(mask, mask, mask=edges)
        # Convert back to 3 channels
        preprocessed_image = cv2.cvtColor(preprocessed_image, cv2.COLOR_GRAY2RGB)
        return preprocessed_image

In [9]:
image = preprocess_image(image)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
image_folder = "/kaggle/input/own-dataset-44img-6kp/own_dataset_6kp"
annotations_path = "/kaggle/input/own-dataset-44img-6kp/own_dataset_6kp/annotations.json"

In [11]:
split_data(image_folder, annotations_path)

NameError: name 'split_data' is not defined

In [ ]:
train_dataset = KeypointsDataset("/kaggle/input/own-dataset-44img-6kp/own_dataset_6kp", "/kaggle/working/data_train.json")
image_folder = '/kaggle/input/own-dataset-44img-6kp/own_dataset_6kp'
train_annotations_path = '/kaggle/working/data_train.json'
val_annotations_path = '/kaggle/working/data_val.json'

train_loader, val_loader = create_dataloaders(image_folder, train_annotations_path, val_annotations_path, batch_size=4, preprocess=True)


# Creat model

In [ ]:
# Define the path to the downloaded weights file
weights_path = '/kaggle/input/resnet-50/pytorch/resnet-50/1/resnet50-0676ba61.pth'

# # Load the model without pre-trained weights
# model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Load the pre-trained weights from the file with map_location to handle CPU-only environments
model = models.resnet50()
model.load_state_dict(torch.load(weights_path, map_location=device))

# Modify the fully connected layer to match the number of keypoints
num_keypoints = 6  # Example number of keypoints
model.fc = nn.Linear(model.fc.in_features, num_keypoints * 2)  # For (x, y) coordinates

# Move the model to the appropriate device
model = model.to(device)



# Train model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 200

In [ ]:
for epoch in range(epochs):
    model.train()
    for i, batch in enumerate(train_loader):
        imgs = batch['image'].to(device)
        kps = batch['keypoints'].view(batch['keypoints'].size(0), -1).to(device)  # Flatten target keypoints
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            imgs = batch['image'].to(device)
            kps = batch['keypoints'].view(batch['keypoints'].size(0), -1).to(device)  # Flatten target keypoints

            outputs = model(imgs)
            loss = criterion(outputs, kps)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch}, Val Loss: {val_loss}")

In [ ]:
torch.save(model.state_dict(), "keypoints_model_weights.pth")